In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_profiling as pp
import seaborn as sns
import warnings
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
pwd

In [ ]:
df =pd.read_excel('Empresas_Reactiva_Perú.xlsx')
df = pd.DataFrame(df)
df.head(n=10) 
df.info()

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
df1 = pd.read_excel('Income.xlsx')
df1 = pd.DataFrame(df1)
df1.head(n=10) 
df1.info()

In [ ]:
df1.head(3)

In [ ]:
df2 = pd.merge(df, df1, on='RUC')

In [ ]:
df2.head(3)

In [ ]:
#CREAMOS EL % DE COBERTURA:

df2['PORCENTAJE_COBERTURA'] = df2['MONTO_COBERTURA'] / df2['MONTO_PRESTAMO']

In [ ]:
df2.info()

In [ ]:
df2.head(3)

In [ ]:
corr = df2[['MONTO_PRESTAMO','MONTO_COBERTURA','INCOME','PORCENTAJE_COBERTURA']].corr().round(4)
corr

In [ ]:
X=np.array(df2[['PORCENTAJE_COBERTURA','MONTO_PRESTAMO','INCOME']])
X.shape

In [ ]:
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Número de Grupos')
plt.ylabel('Puntaje')
plt.title('Curva de Elbow')
plt.show()

In [ ]:
num_clusters = 4

In [ ]:
data=df2[['PORCENTAJE_COBERTURA','MONTO_PRESTAMO','INCOME']]

In [ ]:
#Segmentación de KMEANS

kmeans = KMeans(init='k-means++', n_clusters=num_clusters, n_init=10)
kmeans.fit(data)

In [ ]:
def form_cluster(x,k):
    no_clusters = k
    model = KMeans(n_clusters=no_clusters,init='random')    
    model.fit(x)    
    labels = model.labels_ 
    print(labels)    
    # Cacluate the silhouette score    
    sh_score = silhouette_score(x,labels)    
    return sh_score

In [ ]:
prediction = kmeans.predict(data)

In [ ]:
prediction

In [ ]:
pronosseg = pd.DataFrame(prediction)

In [ ]:
pronosseg.columns=['Segmento']

In [ ]:
sns.pairplot(pd.concat([data,pronosseg],axis=1), hue="Segmento")

In [ ]:
res1=pd.concat([data,pronosseg],axis=1).groupby(['Segmento']).mean()
res1.style.applymap(lambda x: 'background-color : yellow' if x>res1.iloc[0,0] else '')

In [ ]:
pronosseg.head()

In [ ]:
data_final=pd.concat([df2,pronosseg],axis=1)

In [ ]:
data_final.head(3)

In [ ]:
#EXPORTAR BASE

data_final.to_csv("resultados_test_2.csv",index=False)